In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

Using backend: pytorch


# data

In [2]:
import dgl.data

In [3]:
dataset = dgl.data.CoraGraphDataset()

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [4]:
g = dataset[0]

In [5]:
#node feat
g.ndata

{'feat': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0526, 0.0000]]), 'label': tensor([4, 4, 4,  ..., 4, 3, 3]), 'test_mask': tensor([ True,  True, False,  ..., False, False, False]), 'val_mask': tensor([False, False,  True,  ..., False, False, False]), 'train_mask': tensor([False, False, False,  ..., False, False, False])}

In [6]:
#edge feat
g.edata

{}

# model

In [7]:
from dgl.nn import GraphConv

In [8]:
GraphConv??

In [9]:
class GCN(nn.Module):
    def __init__(self,in_feats,h_feats,num_classes):
        super(GCN,self).__init__()
        self.gc1 = GraphConv(in_feats=in_feats,out_feats=h_feats)
        self.gc2 = GraphConv(in_feats=h_feats,out_feats=num_classes)
    
    def forward(self,g,in_feat):
        h = self.gc1(g,in_feat)
        h = F.relu(h)
        h = self.gc2(g,h)
        return h

In [10]:
model = GCN(g.ndata['feat'].shape[1],16,dataset.num_classes)

In [11]:
model

GCN(
  (gc1): GraphConv(in=1433, out=16, normalization=both, activation=None)
  (gc2): GraphConv(in=16, out=7, normalization=both, activation=None)
)

In [12]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [13]:
loss_func = torch.nn.CrossEntropyLoss()

# train

In [14]:
model = model.to('cuda')

In [15]:
def train(g,model):
    best_val_acc = 0
    best_test_acc = 0
    
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    
    for e in range(500):
        logits = model(g,features)
        pred = logits.argmax(1)
        
        ##
        loss = loss_func(logits[train_mask],labels[train_mask])
        
        train_acc = (pred[train_mask]==labels[train_mask]).float().mean()
        val_acc = (pred[val_mask]==labels[val_mask]).float().mean()
        test_acc = (pred[test_mask]==labels[test_mask]).float().mean()
        
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc  = test_acc
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if e%5 == 0:
            print('epoch:%d,loss:%f,val_acc:%f -best_val:%f,test_acc:%f - best_test:%f,'%(e,\
                                                                                           loss,val_acc,best_val_acc,test_acc,best_test_acc))

In [16]:
train(g.to("cuda"),model)

epoch:0,loss:1.947057,val_acc:0.080000 -best_val:0.080000,test_acc:0.084000 - best_test:0.084000,
epoch:5,loss:1.905572,val_acc:0.488000 -best_val:0.488000,test_acc:0.485000 - best_test:0.485000,
epoch:10,loss:1.833501,val_acc:0.594000 -best_val:0.596000,test_acc:0.571000 - best_test:0.572000,
epoch:15,loss:1.734872,val_acc:0.644000 -best_val:0.644000,test_acc:0.614000 - best_test:0.614000,
epoch:20,loss:1.610127,val_acc:0.678000 -best_val:0.678000,test_acc:0.676000 - best_test:0.676000,
epoch:25,loss:1.460230,val_acc:0.690000 -best_val:0.690000,test_acc:0.702000 - best_test:0.702000,
epoch:30,loss:1.291918,val_acc:0.712000 -best_val:0.712000,test_acc:0.719000 - best_test:0.719000,
epoch:35,loss:1.112895,val_acc:0.718000 -best_val:0.720000,test_acc:0.728000 - best_test:0.719000,
epoch:40,loss:0.934309,val_acc:0.726000 -best_val:0.726000,test_acc:0.735000 - best_test:0.735000,
epoch:45,loss:0.766646,val_acc:0.738000 -best_val:0.738000,test_acc:0.747000 - best_test:0.744000,
epoch:50,los

epoch:430,loss:0.003155,val_acc:0.776000 -best_val:0.778000,test_acc:0.752000 - best_test:0.760000,
epoch:435,loss:0.003085,val_acc:0.774000 -best_val:0.778000,test_acc:0.752000 - best_test:0.760000,
epoch:440,loss:0.003017,val_acc:0.774000 -best_val:0.778000,test_acc:0.752000 - best_test:0.760000,
epoch:445,loss:0.002953,val_acc:0.774000 -best_val:0.778000,test_acc:0.752000 - best_test:0.760000,
epoch:450,loss:0.002890,val_acc:0.774000 -best_val:0.778000,test_acc:0.752000 - best_test:0.760000,
epoch:455,loss:0.002828,val_acc:0.774000 -best_val:0.778000,test_acc:0.752000 - best_test:0.760000,
epoch:460,loss:0.002768,val_acc:0.774000 -best_val:0.778000,test_acc:0.751000 - best_test:0.760000,
epoch:465,loss:0.002712,val_acc:0.774000 -best_val:0.778000,test_acc:0.751000 - best_test:0.760000,
epoch:470,loss:0.002657,val_acc:0.774000 -best_val:0.778000,test_acc:0.750000 - best_test:0.760000,
epoch:475,loss:0.002603,val_acc:0.774000 -best_val:0.778000,test_acc:0.750000 - best_test:0.760000,
